# ĐỀ XUẤT LAPTOP DỰA THEO NHU CẦU VÀ THÔNG TIN NGƯỜI DÙNG

# MỤC TIÊU

Ở đề tài này, chúng em sẽ tạo ra một Recommendation System, dựa vào dữ liệu laptop trên thị trường và các số điểm đánh giá. Để đề xuất một danh sách các laptop phù hợp, với input là 1 **phân loại** cho trước (văn phòng, gaming, ...) và một số thông tin của người dùng **(nghề nghiệp, ngành học)**

# ỨNG DỤNG

- Tắng tính hiệu quả trong việc chọn đối tượng của quảng cáo và tư vấn

- Tăng hiệu quả tính năng tìm kiếm cho các trang bán laptop

## TRAIN DATA

Dữ liệu train sẽ được crawl từ 3 nguồn: 

- Tiki (tiki.vn)

- Nguyễn Kim (nguyenkim.com)

- Thế giới di động (thegioididong.com) 

In [34]:
%matplotlib inline
import matplotlib.pyplot as plt  
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.base import TransformerMixin, BaseEstimator
import re
import math

In [35]:
data = pd.read_csv('crawler/Merge.csv')

### Dữ liệu bao gồm các cột:

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 11 columns):
Brand         1163 non-null object
CPU           769 non-null object
Graphic       865 non-null object
Name          1163 non-null object
Price         1163 non-null int64
RAM           877 non-null object
SSD           1163 non-null int64
Score_star    1163 non-null float64
Type_CPU      829 non-null object
Vote          1163 non-null int64
Weight        722 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 100.1+ KB


In [37]:
data.head(5)

,Brand,CPU,Graphic,Name,Price,RAM,SSD,Score_star,Type_CPU,Vote,Weight
0,HP,Intel Core i3,Intel HD Graphics 620,Laptop HP 240 G7 (9FM95PA),8990000,4 GB,0,4.9,7020U,10,NaN
1,Asus,NaN,NaN,Laptop Asus X409UA-EK092T,9390000,4 GB,0,4.4,NaN,10,1.6
2,Asus,Intel Core i5,Intel UHD Graphics 620,Laptop Asus X409FA (EK100T),11590000,4 GB,0,4.5,8265U,10,1.6
3,HP,Core i5,Intel UHD Graphics 620,Laptop HP NoteBook 14S-CF1040TU (7PU14PA),12990000,4 GB,0,4.6,8250U,10,NaN
4,HP,Intel Core i5,Intel HD Graphics 620,Laptop HP 348 G5 (7XU21PA),13990000,4 GB,1,4.5,8265U,10,1.48


## PHƯƠNG PHÁP ĐÁNH GIÁ VÀ CÁCH TIẾN HÀNH

Chúng em sẽ chọn phương pháp **supervised learning** để phần loại (tự cho yếu tố phân loại) và đề xuất (**user-based recommendation**) dựa theo thông tin matching giữa người dùng và sản phẩm.

### Yếu tố phân loại

Chúng em sẽ tạo 2 bảng heuristic để tính điểm cho mỗi sản phẩm - cột **points** là điểm sẽ tăng lên nếu sản phẩm match với **in** đã cho

In [38]:
cate_points = pd.read_csv('categories_points.csv')
major_points = pd.read_csv('major_points.csv')

In [39]:
cate_points.head()

,cate_id,name,field,in,points
0,0,van_phong,Price,"0,4000000",3
1,0,van_phong,Price,"4000000,10000000",2
2,1,gaming,Graphic,"nvidia,geforce,amd",3
3,1,gaming,RAM,"8,16,32",2
4,1,gaming,RAM,4,1


In [40]:
major_points.head()

,name,field,in,points
0,IT,RAM,"4,8,16",1
1,IT,Graphic,"nvidia,geforce,amd",3


### Thuật toán cần chọn

> to be updating

### Tiền xử lý

1. Fill các giá trị trống

In [41]:
fill_string = SimpleImputer(strategy='constant', fill_value='Unknown')

preprocess: Pipeline = make_pipeline(
    fill_string
)

2. Bỏ các chuỗi/đơn vị thừa

In [42]:
class UnitRemoverAndGraphicSimpler(TransformerMixin):
    columns: np.array
        
    def __init__(self, columns: np.array):
        super()
        self.columns = columns
    
    def fit(self, *_):
        return self

    @staticmethod
    def ramConvert(ram):
        ram_str = str(ram)
        ram_str = re.sub(r'\D', '', ram_str)
        if ram_str == '':
            return np.nan
        return int(ram_str)
    
    @staticmethod
    def weightConvert(weight):
        weight_str = re.sub(',', '.', str(weight))
        try:
            return int( math.floor( float(weight_str)))
        except ValueError:
            return np.nan
    
    def transform(self, X_df: np.ndarray, *_) -> pd.DataFrame:
        df = pd.DataFrame(X_df, columns=self.columns)
        df['RAM'] = df['RAM'].apply(self.ramConvert)
        df['Weight'] = df['Weight'].apply(self.weightConvert)
        df['Graphic'] = df['Graphic'].apply(lambda g: g.split(' ')[0].lower())
        df['Brand'] = df['Brand'].apply(lambda b: b.lower())
        
        return df

3. Tính điểm

In [65]:
class CalculatePoints(BaseEstimator, TransformerMixin):
    point_data: pd.DataFrame
    columns: np.array

    def __init__(self, point_data: pd.DataFrame, columns):
        super()
        self.point_data = point_data
        self.columns = columns
        
    def fit(self, *_):
        return self
    
    def calcPrice(self, price: int) -> dict:
        price_data: pd.DataFrame = self.point_data[self.point_data['field'] == 'Price']
        point_out = dict()
        for k, row in price_data.iterrows():
            name = row['name']
            val_in = row['in']
            point = row['points']
            [val_min, val_max] = val_in.split(',')
            if int(val_min) <= price and int(val_max) >= price:
                point += point_out.get(name, 0)
                point_out[name] = point
                
        return point_out
       
    def calcInArray(self, value) -> dict:
        point_out = dict()
        for k, row in self.point_data.iterrows():
            name = row['name']
            val_in = row['in']
            point = row['points']
            array_val = val_in.split(',')
            if value in array_val:
                point += point_out.get(name, 0)
                point_out[name] = point
                
        return point_out

    def transform(self, data: np.ndarray):
        point_names = self.point_data['name'].unique()
        point_columns = self.point_data['field'].unique() 
        print(point_names)
        
        df = pd.DataFrame(data, columns=self.columns)
        add_cols = dict()
        
        for col in point_columns:
            func = {
                "Price": self.calcPrice
            }.get(col, self.calcInArray) # default

            points = df[col].map(func)

            if points is not None:
                for idx, point in points.items():
#                     add_cols = add_cols.add(point)
                    
#                     add_cols = { key: add_cols.get(key, 0) + point.get(key, 0) 
#                                 for key in set(add_cols) | set(point)  }
# #                     print(f"DEBUG: {idx} - {point}")
                    for k in point_names:
                        old_arr = add_cols.get(k, [])
                        if len(old_arr) <= idx:
                            old_arr.append(point.get(k, 0))
                        else:
                            old_arr[idx] += point.get(k, 0)
                        add_cols[k] = old_arr
                        
        return add_cols

### Chuẩn hóa vào 1 pipeline

In [66]:
pipeline = Pipeline([
    ('preprocess', preprocess),
    ('unit_remove', UnitRemoverAndGraphicSimpler(columns=data.columns)),
    ('fill_nan', SimpleImputer(strategy='most_frequent')),
    ('calc_point', CalculatePoints(point_data=major_points, columns=data.columns))
])

In [67]:
pipeline.fit(data)
# preprocess.fit(data)
# preprocess.fit_transform(data)[0]
# fill_string_pipeline.fit_transform(data)[0]

Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=None,
                          steps=[('simpleimputer',
                                  SimpleImputer(add_indicator=False, copy=True,
                                                fill_value='Unknown',
                                                missing_values=nan,
                                                strategy='constant',
                                                verbose=0))],
                          verbose=False)),
                ('unit_remove',
                 <__main__.UnitRemoverAndGraphicSimpler object at 0x7fb33b06e470>),
                ('fill_nan',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='most_frequent',
                               verbose=0)),
                ('calc_point',
                 CalculatePoints(columns=Index(['Brand', 'CPU', 'Graphic', 'Name', 'Pric

### TRAIN DATA

> updating

In [68]:
# data.head(1)
points = pipeline.transform(data)

['IT']


In [69]:
# print(points)
# print(len(points['van_phong']))
print(data.shape)

(1163, 11)


In [70]:
train_output = data.copy()
for k, v in points.items():
    train_output[f'{k}_points'] = pd.Series(v, index=train_output.index)
    
train_output.to_csv('points.csv', index=False)

## KIỂM TRA VÀ DỰ ĐOÁN

> updating

In [76]:
train_output.sort_values(by=['IT_points', 'Score_star', 'Vote'], ascending=False).head(10)

,Brand,CPU,Graphic,Name,Price,RAM,SSD,Score_star,Type_CPU,Vote,Weight,IT_points
233,MSI,Intel Core i7,NVIDIA GeForce GTX 1050Ti 4GB GDDR5 + Intel UH...,Laptop MSI GF63 Thin 9RCX-645VN Core i7-9750H/...,22039000,8GB,1,5.0,9750H,4,1.86,3
288,Asus,Intel Core i5,NVIDIA GeForce MX150 + Intel UHD Graphics 620,Laptop Asus Vivobook S15 S530FN-BQ133T Core i5...,17219000,4GB,1,5.0,8265U,4,1.8,3
266,Asus,AMD Ryzen,NVIDIA GeForce GTX 1050 4GB GDDR5,Laptop Asus F570ZD-FY415T AMD R5-2500U/ GTX 10...,15190000,8GB,1,5.0,2500U,3,1.55,3
290,Asus,AMD Ryzen 7-3750H,NVIDIA GeForce GTX 1050,Laptop Asus TUF Gaming FX705DD-AU059T AMD R7-3...,22990000,8GB,1,5.0,3750H,3,2.6,3
296,Acer,Intel Core i7,NVIDIA GeForce GTX 1060,Laptop Acer Predator Helios 300 PH315-51-7533 ...,25149000,8GB,1,5.0,8750H,2,2.7,3
347,Acer,Intel Core i7,NVIDIA GeForce GTX1050Ti,Laptop Acer Nitro 5 AN515-52-75FT NH.Q3LSV.003...,22990000,8GB,1,5.0,8750H,2,2.7,3
271,MSI,NaN,NVIDIA GeForce GTX 1650 4GB GDDR5 with Max-Q D...,Laptop Gaming MSI GF63 Thin 9SC-070VN Core i7-...,24429000,"8GB DDR4 2666MHz (2x SO-DIMM socket, up to 32G...",1,5.0,Intel Core i7-9750H (2.6GHz up to 4.5GHz 12MB),1,1.86,3
275,Asus,AMD Ryzen 7-3750H,NVIDIA GeForce GTX 1660Ti,Laptop Asus ROG Zephyrus G GA502DU-AL024T AMD ...,28799000,8GB,1,5.0,3750H,1,2.1,3
289,Asus,NaN,Nvidia geforce MX250 2GB GDDR5 + Intel UHD Gra...,Laptop Asus Vivobook S531FL-BQ192T Core i7-856...,21749000,8GB DDR4 2400MHz,1,5.0,"Intel Core i7-8565U (1.8GHz upto 4.6GHz, 4Core...",1,NaN,3
341,Asus,Intel Core i5,NVIDIA GeForce GTX 1050Ti + Intel UHD Graphics...,Laptop ASUS TUF Gaming FX505GE-BQ052T Core i5-...,21389000,8GB,1,5.0,8300H,1,2.2,3
